# From Amazon Exercise
https://docs.aws.amazon.com/vpc/latest/userguide/vpc-subnets-commands-example.html

# boto documentation for EC2
https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html?highlight=routetable


In [2]:
import boto3

In [3]:
ec2 = boto3.resource('ec2', region_name='us-west-2')

aws ec2 create-vpc --cidr-block 10.0.0.0/16 --query Vpc.VpcId --output text

In [8]:
vpc = ec2.create_vpc(CidrBlock='10.0.0.0/16')

In [10]:
vpc.id

'vpc-0708b5747ee571199'

# Creating 2 subnets in the VPC created above
- aws ec2 create-subnet --vpc-id vpc-2f09a348 --cidr-block 10.0.1.0/24
- aws ec2 create-subnet --vpc-id vpc-2f09a348 --cidr-block 10.0.0.0/24


In [12]:
subnet_0 = ec2.create_subnet(VpcId=vpc.id, CidrBlock='10.0.0.0/24' )

In [13]:
subnet_1 = ec2.create_subnet(VpcId=vpc.id, CidrBlock='10.0.1.0/24' )

# Creating an internet gateway to allow external traffic
aws ec2 create-internet-gateway --query InternetGateway.InternetGatewayId --output text

In [16]:
gw = ec2.create_internet_gateway()

ec2.InternetGateway(id='igw-0656a5e09e0ffd725')

In [18]:
gw = ec2.InternetGateway('igw-0656a5e09e0ffd725')

In [20]:
gw

ec2.InternetGateway(id='igw-0656a5e09e0ffd725')

# Attach Gateway to VPC
aws ec2 attach-internet-gateway --vpc-id vpc-2f09a348 --internet-gateway-id igw-1ff7a07b

In [25]:
response = gw.attach_to_vpc(VpcId = vpc.id)
response

{'ResponseMetadata': {'RequestId': '2b3b24ae-2cc3-4029-8aa4-dd627f757469',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2b3b24ae-2cc3-4029-8aa4-dd627f757469',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '243',
   'date': 'Sun, 05 Feb 2023 13:55:11 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# Create a route table "In a VPC" to allow all internet traffic to a gateway and then to a subnet 
aws ec2 create-route-table --vpc-id vpc-2f09a348 --query RouteTable.RouteTableId --output text


In [27]:
route_table = ec2.create_route_table(VpcId=vpc.id)

In [28]:
route_table

ec2.RouteTable(id='rtb-08f5bebb85b644da0')

# Create a route that points all traffic to the internet gateway 
aws ec2 create-route --route-table-id rtb-c1c8faa6 --destination-cidr-block 0.0.0.0/0 --gateway-id igw-1ff7a07b

In [32]:
route = route_table.create_route(DestinationCidrBlock='0.0.0.0/0', GatewayId=gw.id)

In [38]:
route

ec2.Route(route_table_id='rtb-08f5bebb85b644da0', destination_cidr_block='0.0.0.0/0')

# Associsate route to one of the subnets created above (subnet_1)
aws ec2 associate-route-table  --subnet-id subnet-b46032ec --route-table-id rtb-c1c8faa6

In [41]:
assoc = route_table.associate_with_subnet(SubnetId=subnet_1.id)
assoc

ec2.RouteTableAssociation(id='rtbassoc-01ee9f21df42b9926')

In [4]:
gw = ec2.InternetGateway('igw-0656a5e09e0ffd725')

In [5]:
route_table = ec2.RouteTable('rtb-08f5bebb85b644da0')

In [6]:
vpc = ec2.Vpc('vpc-0708b5747ee571199')